<a href="https://cognitiveclass.ai/">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Logo/SNLogo.png" width="200" align="center">
</a>


<h1>Lab - Training Custom Classifiers with IBM Watson Visual Recognition in Python</h1>

<h2>Introduction</h2>

## Objectives

After completing this lab you will be able to:

-   How to operate the Watson Visual Recognition API and OpenCV using the Python Programming Language
-   Understand the advantage of using the Watson Visual Recognition API over the Graphic User Interface on the Browser
-   How to automate the training, and testing of your Visual Recognition model.

<p>In this lab you will be training a Visual Recognition model that classify different kinds of dogs by running python code.</p>


<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size="3"><strong>Click on the links to go to the following sections:</strong></font>
<br>
<h2>Table of Contents</h2>
<ol>
    <li><a href="#ref1">IBM Watson Package</a></li>
    <li><a href="#ref2">Setting the API key for IBM Watson Visual Recognition</a></li>
    <li><a href="#ref3">Training the Classifier</a></li>
    <li><a href="#ref4">Testing the Classifier</a></li>
    <li><a href="#ref5">Exercises</a></li>
</ol>    
</div>


<a id="ref1"></a>

<h2>IBM Watson Package</h2>
In order to run this lab we need to import the following package.
<ul>
    <li>IBM Watson: which allows access to the Watson Visual Recognition API</li>
</ul>
The code below will install IBM Watson. 

To run, click on the code cell below and press "shift + enter".

<b>NOTE - The Watson Developer Cloud Package has been deprecated and has been replaced by the IBM Watson Package </b>


In [3]:
!pip install --upgrade ibm-watson

Requirement already up-to-date: ibm-watson in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (4.7.1)


<h3>Goal of this lab:</h3>


<p>In this lab, we will be creating a completely new image classifier using training images. We will train a custom classifier to identify between three different dog breeds (Golden Retriever, Beagle and Husky).</p>


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/dog-breed.png" width="480"/>


<a id="ref2"></a>

<h2>Setting the API key for IBM Watson Visual Recognition</h2>

<p>In order for you to use the IBM Watson Visual Recognition API, you will need the API key of the Visual Recognition instance that you have created in the previous sections.</p>

<p>Log into your IBM Cloud Account with the following link.</p> <a href="https://cocl.us/CV0101EN_IBM_Cloud_Login">https://cloud.ibm.com</a>
<ol>
    <li>Click on <b>Services</b></li>
    <li>Under Services, click on your Watson Visual Recognition Instance</li>
    <li>Copy the <b>API Key</b> and past it in the code cell below</li>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/API_Key.png" width="680">
    <li>Then press "ctrl + enter" to run the code cell.</li>
</ol>


In [5]:
# Paste your API key for IBM Watson Visual Recognition below:
my_apikey = '32njUe7Nh7DC4DpUkVMQv0XGWOPetg2Xgn4pRqZl3uXr'

<h4>Initialize Watson Visual Recognition</h4>
Lets create your own Watson Visual Recognition instance, it will allow you to make calls to the Watson Visual Recognition API.


In [6]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2018-03-19', 
                             authenticator=authenticator)


<p>We are going to train an Image Recognition model to classify different types of dog. The dataset that we are going to use are the zip files that we use below</p>


<ul>
    <li>beagle.zip</li>
    <li>husky.zip</li>
    <li>golden-retriever.zip</li>
</ul>


<a id="ref3"></a>

<h2>Training Classifier</h2>


<h4>Download the differerent breed of dog images as zip files</h4>
<p>We will use the <b>urlretrieve</b> method from the <b>urllib.request</b> library to download the dataset above.</p> 


In [4]:
import urllib.request

# Downloading Beagle dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Beagle.zip", 
                           "beagle.zip")

# Downloading Husky dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Husky.zip", 
                           "husky.zip")

# Downloading Golden Retriever dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever.zip", 
                           "goldenretriever.zip") #note that we should remove any hyphens from the zip file name

('goldenretriever.zip', <http.client.HTTPMessage at 0x7f80d8452320>)

<p>Lets train our Visual Recognition model to recognize the three breeds of dogs using the <b>create_classifier</b> method from the Watson Image Recognition API.</p>


In [5]:
import json
with open('beagle.zip', 'rb') as beagle, \
     open('goldenretriever.zip', 'rb') as gretriever, \
     open('husky.zip', 'rb') as husky:
        response = visrec.create_classifier(name="dogbreedclassifier",
                                        positive_examples={'beagle': beagle, \
                                                           'goldenretriever': gretriever, \
                                                           'husky': husky})
print(json.dumps(response.get_result(), indent=2))

{
  "classifier_id": "dogbreedclassifier_1049415485",
  "name": "dogbreedclassifier",
  "status": "training",
  "owner": "859b012c-94ca-4269-8bde-30e91a56eb30",
  "created": "2020-10-03T22:32:34.647Z",
  "updated": "2020-10-03T22:32:34.647Z",
  "classes": [
    {
      "class": "husky"
    },
    {
      "class": "goldenretriever"
    },
    {
      "class": "beagle"
    }
  ],
  "rscnn_enabled": false,
  "core_ml_enabled": true
}


In [6]:
#lets grab the classifier id
classifier_id = response.get_result()["classifier_id"]
classifier_id

'dogbreedclassifier_1049415485'

<div style="background-color: #fcf2f2">
    <h2>Note!</h2> 
    <p>If you receive the following error.</p>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/Error-Code.png" width="1080">
    <p>It means that you have more than 1 Visual Recognition Instances running on your lite plan, and the lite plan only allows for no more than 2 Visual Recognition instances. So you might want to delete one of your custom classifier in your Watson Visual Recognition Instance.</p>
        <p>Log into your IBM Cloud Account with the following link.</p> <p><a href="https://cocl.us/CV0101EN_IBM_Cloud_Login">https://cloud.ibm.com</a></p>
    <ol>
        <li>Click on <b>Services</b></li>
        <li>Under Services, click on your Watson Visual Recognition Instance</li>
        <li>Then click on Create a Custom Model</li>
        <li>Then delete one of your Custom Visual Recognition Model</li>
    </ol>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/Delete-Instance.png" width="680">
</div>
 


<h4>Is the model still training?</h4>
Depending on the number of images, it may take <b>a Several Minutes </b> for Watson to build a custom classifier. Please wait tell you get <b> Good to Go<b> 


In [14]:
Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

Good to go 


<h4>List all (custom) classifiers</h4>


<h4>If the status is still training, please rerun the above cell and wait until you see ready</h4> 


In [8]:
visrec.list_classifiers(verbose=True).get_result()

{'classifiers': [{'classifier_id': 'dogbreedclassifier_1049415485',
   'name': 'dogbreedclassifier',
   'status': 'training',
   'owner': '859b012c-94ca-4269-8bde-30e91a56eb30',
   'created': '2020-10-03T22:32:34.647Z',
   'updated': '2020-10-03T22:32:34.647Z',
   'classes': [{'class': 'husky'},
    {'class': 'goldenretriever'},
    {'class': 'beagle'}],
   'rscnn_enabled': False,
   'core_ml_enabled': True}]}

<a id="ref4"></a>

<h2>Testing Classifier</h2>
<p>Let's test the classifier, the function <b>getdf_visrec</b> below uses the method <b>classify</b> from Watson Visual Recognition API to upload the image to the classifier and give us a result in JSON(JavaScript Object Notation) format. Then we use the method <b>json_normalize</b> from the "Pandas" library in Python to turn the result into a table because it is more human readable.</p>


In [12]:
from pandas.io.json import json_normalize

def getdf_visrec(url, classifier_ids, apikey = my_apikey):
    
    json_result = visrec.classify(url=url,
                              threshold='0.6',
                              classifier_ids=classifier_id).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever1_stacked.jpg">
<p>Let's test our Visual Recognition model on this picture of Golden Retriever</p>


<p>Please wait for your Custom Model to finish training before you upload your test image to your Custom Classifier, <b>you might get an error if your model is still training and you run the function below.</p>
<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/Training.png" width="680">


In [15]:
getdf_visrec(url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever1_stacked.jpg',
            classifier_ids=classifier_id)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,class,score
0,goldenretriever,0.908


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/cat-2083492_960_720.jpg">
<p>Lets test our Visual Recognition model on this picture of cat</p>


In [16]:
getdf_visrec(url = 'http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/cat-2083492_960_720.jpg',
            classifier_ids=classifier_id)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,class,score
0,husky,0.821


<p>Our model will mis-classify the cat in the picture because our custom visual recognition model is only trained for recognizing different breeds of dogs.</p>


<h4>Delete all classifiers</h4>
<p>If you want to delete you classifiers, lets get the classifier id tht you want to delete. The method <b>list_classifiers</b> from Watson Visual Recognition API list all the classifier in your IBM Cloud account.</p>


In [17]:
import json

classifiers = visrec.list_classifiers(verbose=True).get_result()['classifiers']
print(json.dumps(classifiers, indent=2))

[
  {
    "classifier_id": "dogbreedclassifier_1049415485",
    "name": "dogbreedclassifier",
    "status": "ready",
    "owner": "859b012c-94ca-4269-8bde-30e91a56eb30",
    "created": "2020-10-03T22:32:34.647Z",
    "updated": "2020-10-03T22:32:34.647Z",
    "classes": [
      {
        "class": "husky"
      },
      {
        "class": "goldenretriever"
      },
      {
        "class": "beagle"
      }
    ],
    "rscnn_enabled": false,
    "core_ml_enabled": true
  }
]


Just paste your classifier id and it will be deleted


In [18]:
mycid = 'dogbreedclassifier_1049415485' #the classifier id you want to delete
visrec.delete_classifier(classifier_id = mycid)

<a id="ref5"></a>

<h2>Exercises</h2>
<p>For the following exercises you are going to train a Custom Visual Recognition Classifier to recognize fast food items, in particular it will be able to classify food items into <b>Burger</b>, <b>Fries</b> or <b>Coke</b>.<p>


<h3>Question 1: Optional</h3>
<p>After training your custom classifier, you might want to delete all the custom classifiers from your account, write a piece of code to use the *list_classifier* and *delete_classifier* method to delete all the custom classifiers in your account.</p>

<p><b>Note!</b> This question is optional, if there is a classifier that you have trained and do not want to delete, skip this question.</p>


In [19]:
# Write your code below and press Shift+Enter to execute 
# The code below will delete all classifiers in your account
for i in visrec.list_classifiers().get_result()['classifiers']:
    print('Deleting ...' + i['classifier_id'])
    visrec.delete_classifier(classifier_id=i['classifier_id'])

Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:

# The code below will delete all classifiers in your account
for i in visrec.list_classifiers().get_result()['classifiers']:
    print('Deleting ...' + i['classifier_id'])
    visrec.delete_classifier(classifier_id=i['classifier_id'])
-->


<h3>Question 2</h3>
<p>The link to the data set for Burger, Fries and Coke is given below</p> 
    
<p>We will use the <b>urlretrieve</b> method from the <b>urllib.request</b> library to download the dataset below.</p> 

<ul>
    <li>https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Burger.zip</li>
    <li>https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Fries.zip</li>
    <li>https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Pizza.zip</li>
</ul>


In [8]:
# Write your code below and press Shift+Enter to execute 

import urllib.request

# Downloading Beagle dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Burger.zip",
                          "Burger.zip")

# Downloading Husky dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Fries.zip", 
                           "Fries.zip")

# Downloading Golden Retriever dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Pizza.zip", 
                           "Pizza.zip") #note that we should remove any hyphens from the zip file name

('Pizza.zip', <http.client.HTTPMessage at 0x7f91aa567358>)

Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
import urllib.request

#Downloading Burger dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Burger.zip", 
                           "burger.zip")

#Downloading Fries dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Fries.zip", 
                           "fries.zip")

#Downloading Pizza dataset
urllib.request.urlretrieve("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Pizza.zip", 
                           "pizza.zip")
-->


<h3>Question 3.1</h3>
<p>Now we have the dataset, use the <b>create_classifier</b> method to create your fast food classifier.</p>


In [15]:
# Write your code below and press Shift+Enter to execute 
import json
from ibm_watson import VisualRecognitionV3
my_apikey = '32njUe7Nh7DC4DpUkVMQv0XGWOPetg2Xgn4pRqZl3uXr'
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2018-03-19', 
                             authenticator=authenticator)


with open('Burger.zip', 'rb') as burger, \
     open('Fries.zip', 'rb') as fries, \
     open('Pizza.zip', 'rb') as pizza:
        response = visrec.create_classifier(name="fastfoodclassifier",
                                        positive_examples={'burger': burger, \
                                                           'fries': fries, \
                                                           'pizza': pizza})
print(json.dumps(response.get_result(), indent=2))

ERROR:root:{'code': 400, 'error_id': 'input_error', 'description': 'Cannot execute learning task.  : this plan instance can have only 2 custom classifier(s), and 2 already exist.'}
Traceback (most recent call last):
  File "/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 225, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: {'code': 400, 'error_id': 'input_error', 'description': 'Cannot execute learning task.  : this plan instance can have only 2 custom classifier(s), and 2 already exist.'}, Code: 400 , X-global-transaction-id: 0b025459-f8da-40ac-aced-8ca762bcd615


ApiException: Error: {'code': 400, 'error_id': 'input_error', 'description': 'Cannot execute learning task.  : this plan instance can have only 2 custom classifier(s), and 2 already exist.'}, Code: 400 , X-global-transaction-id: 0b025459-f8da-40ac-aced-8ca762bcd615

Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
my_apikey = '<paste your api key here>'

from ibm_watson import VisualRecognitionV3

visrec_fast_food = VisualRecognitionV3(version = '2019-01-01', 
                             iam_apikey = my_apikey)

import json
with open('burger.zip', 'rb') as burger, \
     open('fries.zip', 'rb') as fries, \
     open('pizza.zip', 'rb') as pizza:
    new_response = visrec_fast_food.create_classifier(name="fastfoodclassifier",
                                        positive_examples={'burger': burger, \
                                                           'fries': fries, \
                                                           'pizza': pizza})

    print(json.dumps(new_response.get_result(), indent=2))
-->


<h3>Question 3.2</h3>
<p>Since we will need the classifier_id, grab the classifier_id from the response of create_classifier and store it into a variable.</p>


In [ ]:
import json

classifiers = visrec.list_classifiers(verbose=True).get_result()['classifiers']
print(json.dumps(classifiers, indent=2))

# Write your code below and press Shift+Enter to execute 
#lets grab the classifier id
fast_food_classifier_id = response.get_result()["classifier_id"]
fast_food_classifier_id

Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
#lets grab the classifier id
fast_food_classifier_id = new_response.get_result()['classifier_id']
-->


<h3>Question 4</h3>
<p>Get a url of a picture of fast food and use the <b>getdf_visrec</b> function to classify the picture. <b>Before that, please make sure that your model is trained and ready </b>


In [47]:
# Write your code below and press Shift+Enter to check if your model is ready to go 
Status = visrec.get_classifier(classifier_id=fast_food_classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

Good to go 


Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
gStatus = visrec.get_classifier(classifier_id=fast_food_classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")
-->


In [7]:
# Write your code below and press Shift+Enter to execute 
mycid = 'fastfoodclassifier_212618721' #the classifier id you want to delete
visrec.delete_classifier(classifier_id = mycid)

Double-click <font color="red"><b><u>here</b></u></font> for the solution.

<!-- The answer is below:
getdf_visrec(url = 'fast_food_image_url',
            classifier_ids=fast_food_classifier_id)
-->


<h1>Thank you for completing this notebook</h1>
You can read more about Watson Visual Recognition APIs from the following link.
<a href="https://cloud.ibm.com/apidocs/visual-recognition?code=python">https://cloud.ibm.com/apidocs/visual-recognition</a>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Get IBM Watson Studio free of charge!</h2>
    <p><a href="https://cloud.ibm.com/catalog/services/watson-studio"><img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Logo/BottomAd.png" width="750" align="center"></a></p>
</div>


## Author

<a href="https://www.linkedin.com/in/yi-leng-yao-84451275/" target="_blank" >Yi Yao</a>

## Other Contributors

<a href="https://www.linkedin.com/in/nayefaboutayoun/" target="_blank">Nayef Abou Tayoun</a> 

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-08-27        | 2.0     | Anamika    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
